# **1. Imports**

In [299]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.impute
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight

# **2. train 데이터 전처리**

In [301]:
train = pd.read_csv('train.csv')

## **2-A. train 불필요한 컬럼삭제**

In [302]:
# 변동성이 없는 데이터 제거 (분산 0)
columns_to_drop1 = [col for col in train.columns if train[col].nunique() == 1]

# 결측치 비율이 90% 이상인 열 제거
missing_ratio = train.isnull().mean()
threshold = 0.9
columns_to_drop2 = missing_ratio[missing_ratio >= threshold].index.tolist()

# 중복되는 컬럼 삭제 확실
remove1=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2']
remove2=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2']

remove_columns = columns_to_drop1 + columns_to_drop2 + remove1 + remove2

train = train.drop(columns=remove_columns)

## **2-B. 결측치 처리**
- 랜덤포레스트를 사용할 것이니 -999로 대체해준다.

In [303]:
# 결측치 있는 컬럼 찾기
missing = train.isnull().sum()
missing_index = [i for i,v in enumerate(missing) if v>0] # 16,86,116번 컬럼에 결측치 발견
missing_index
missing_columns = [train.columns[missing_index[0]],train.columns[missing_index[1]],train.columns[missing_index[2]]]

# OK를 NaN으로 대체
for i in range(3): 
    train[missing_columns[i]] = train[missing_columns[i]].replace('OK', np.nan)

# NaN값을 -999로 대체
for i in range(3): 
    train[missing_columns[i]] = train[missing_columns[i]].replace(np.nan, -999)

# 값들을 float형으로 변환
for col in missing_columns:
    train[col] = train[col].astype(float) 

## **2-C. 상관관계 1.0 이상 컬럼 삭제**

In [304]:
# 다중공선성 일으키는 컬럼 삭제 (상관관계 1.0이상, 연속형 변수)
corr_matrix = train.select_dtypes(include=[float, int]).corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

to_drop = [column for column in upper.columns if any(upper[column] >= 1.0)]

train = train.drop(columns=to_drop)
train = train.drop('CURE END POSITION X Collect Result_Dam',axis=1) # 여기서는 상관관계를 계산하지 않은 상관관계가 1인 다른 범주형 변수가 존재함. 그것과의 충돌을 막기 위해 이 컬럼도 삭제함

## **2-D. 피처 엔지니어링**

In [305]:
# (1) num이 다르면 항상 비정상이 나오는 패턴 확인
train['Dam_num'] = [l.split()[-1][-1] for l in train['Equipment_Dam']]
train['Fill1_num'] = [l.split()[-1][-1] for l in train['Equipment_Fill1']]
train['Fill2_num'] = [l.split()[-1][-1] for l in train['Equipment_Fill2']]

train['Num_Diff'] = ((train['Dam_num'] != train['Fill1_num']) | 
                     (train['Dam_num'] != train['Fill2_num']) |
                     (train['Fill1_num'] != train['Fill2_num']))
train = train.drop(['Dam_num','Fill1_num','Fill2_num'],axis=1)

# (2) collect result의 값이 다르면 항상 비정상이 나오는 패턴 확인
train['is_abnormal_pattern'] = (train['Production Qty Collect Result_Fill1'] != train['Production Qty Collect Result_Dam']) | \
                               (train['Production Qty Collect Result_Fill1'] != train['Production Qty Collect Result_Fill2']) | \
                               (train['Production Qty Collect Result_Dam'] != train['Production Qty Collect Result_Fill2'])

# (3) Receip No이 값이 다르면 항상 비정상이 나오는 패턴 확인
train['is_abnormal_pattern2'] = (train['Receip No Collect Result_Dam'] != train['Receip No Collect Result_Fill1']) | \
                               (train['Receip No Collect Result_Dam'] != train['Receip No Collect Result_Fill1']) | \
                               (train['Receip No Collect Result_Fill1'] != train['Receip No Collect Result_Fill2'])

# (4) Resin의 출사 속도와 시간의 곱을 이용해서 얼마나 레진이 나왔는지 새로운 컬럼 제작 -> 레진이 적게 나올 수록 비정상 비율이 높음
train['multi'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']

# (5) 온도와 압력의 곱이 특정 값일 때 항상 비정상
train['temp_pressure_interaction'] = (train['Chamber Temp. Collect Result_AutoClave'] * train['1st Pressure Collect Result_AutoClave'])
train['abnormal_interaction'] = train['temp_pressure_interaction'].apply(lambda x: False if x == 9.952 or x == 10.404 else True)

# (6) 특정 컬럼에서의 비정상 패턴 발견
train['Is_Abnormal_WorkMode'] = (train['WorkMode Collect Result_Fill2'] == 3)
train['Pressure_Below_75'] = (train['3rd Pressure Unit Time_AutoClave'] < 75)
train['Suffix_Dam_AJX75334503'] = (train['Model.Suffix_Dam'] == 'AJX75334503')

## **2-E. int로 되어있지만 사실은 범주형 -> object형으로 바꾸는 과정**

In [306]:
train[['WorkMode Collect Result_Fill2','WorkMode Collect Result_Fill1']] = train[['WorkMode Collect Result_Fill2','WorkMode Collect Result_Fill1']].astype('object')

## **2-F. 랜덤포레스트 학습을 위한 원핫인코딩**

In [307]:
# step1. 범주형 자료만 뽑기 (target 제외)
categorical_cols = train.select_dtypes(include='object').columns.drop('target')

# step2. 원핫 인코딩
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = onehot_encoder.fit_transform(train[categorical_cols])
encoded_df = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(categorical_cols))

# step3. 나머지 데이터프레임과 concat
# 원핫 인코딩된 데이터와 target 열을 분리
train_non_categorical = train.drop(categorical_cols,axis=1)
train_target = train_non_categorical.pop('target')

# step3-1. target 열을 0,1으로 레이블링
dct = {'Normal':0 , 'AbNormal':1}
train_target = train_target.map(dct)

# step4. 원핫 인코딩된 데이터프레임과 나머지 데이터프레임을 합치기
train = pd.concat([train_non_categorical, encoded_df, train_target], axis=1)

## **2-G. 특정 조건, 특정 값에서 항상 비정상을 보이는 것들 추가**

In [308]:
conditions = [
    ('Chamber Temp. Unit Time_AutoClave', 183),
    ('Chamber Temp. Unit Time_AutoClave', 180),
    ('Workorder_Dam_3KPXX094-0001', 1.0),
    ('Workorder_Dam_4CPXX084-0001', 1.0)
]

# 각 조건에 대해 새로운 열 추가
for col, value in conditions:
    condition_name = f'Condition_{col}_{value}'
    train[condition_name] = (train[col] == value).astype(int)

# **3. test 데이터 전처리**

In [330]:
test = pd.read_csv('test.csv')

## **3-A. test 불필요한 컬럼삭제**

In [331]:
# 변동성이 없는 데이터 제거
columns_to_drop1 = [col for col in test.columns if test[col].nunique() == 1]

# 결측치 비율이 90% 이상인 열 제거
missing_ratio = test.isnull().mean()
threshold = 0.9
columns_to_drop2 = missing_ratio[missing_ratio >= threshold].index.tolist()

# 중복되는 컬럼 삭제 확실
remove1=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2']
remove2=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2']

remove_columns = columns_to_drop1 + columns_to_drop2 + remove1 + remove2

test = test.drop(columns=remove_columns)

## **3-B. 결측치 처리**
- 랜덤포레스트를 사용할 것이니 -999로 대체해준다.

In [332]:
# 결측치 있는 컬럼 찾기
missing = test.isnull().sum()
missing_index = [i for i,v in enumerate(missing) if v>0] # 16,86,116번 컬럼에 결측치 발견
missing_index
missing_columns = [test.columns[missing_index[0]],test.columns[missing_index[1]],test.columns[missing_index[2]]]

# OK를 NaN으로 대체
for i in range(3): 
    test[missing_columns[i]] = test[missing_columns[i]].replace('OK', np.nan)

# NaN값을 -999로 대체
for i in range(3): 
    test[missing_columns[i]] = test[missing_columns[i]].replace(np.nan, -999)

# 값들을 float형으로 변환
for col in missing_columns:
    test[col] = test[col].astype(float) 

## **3-C. 상관관계 1.0 이상 컬럼 삭제**

In [ ]:
# 다중공선성 일으키는 컬럼 삭제 (상관관계 1.0이상, 연속형 변수)
corr_matrix = test.select_dtypes(include=[float, int]).corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

to_drop = [column for column in upper.columns if any(upper[column] >= 1.0)]

test = test.drop(columns=to_drop)
test = test.drop('CURE END POSITION X Collect Result_Dam',axis=1) # 여기서는 상관관계를 계산하지 않은 상관관계가 1인 다른 범주형 변수가 존재함. 그것과의 충돌을 막기 위해 이 컬럼도 삭제함

## **3-D. 피처 엔지니어링**

In [ ]:
# (1) num이 다르면 항상 비정상이 나오는 패턴 확인
test['Dam_num'] = [l.split()[-1][-1] for l in test['Equipment_Dam']]
test['Fill1_num'] = [l.split()[-1][-1] for l in test['Equipment_Fill1']]
test['Fill2_num'] = [l.split()[-1][-1] for l in test['Equipment_Fill2']]

test['Num_Diff'] = ((test['Dam_num'] != test['Fill1_num']) | 
                     (test['Dam_num'] != test['Fill2_num']) |
                     (test['Fill1_num'] != test['Fill2_num']))
test = test.drop(['Dam_num','Fill1_num','Fill2_num'],axis=1)

# (2) collect result의 값이 다르면 항상 비정상이 나오는 패턴 확인
test['is_abnormal_pattern'] = (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Dam']) | \
                               (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Fill2']) | \
                               (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill2'])

# (3) Receip No이 값이 다르면 항상 비정상이 나오는 패턴 확인
test['is_abnormal_pattern2'] = (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill1']) | \
                               (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill1']) | \
                               (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Fill2'])

# (4) Resin의 출사 속도와 시간의 곱을 이용해서 얼마나 레진이 나왔는지 새로운 컬럼 제작 -> 레진이 적게 나올 수록 비정상 비율이 높음
test['multi'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']

# (5) 온도와 압력의 곱이 특정 값일 때 항상 비정상
test['temp_pressure_interaction'] = (test['Chamber Temp. Collect Result_AutoClave'] * test['1st Pressure Collect Result_AutoClave'])
test['abnormal_interaction'] = test['temp_pressure_interaction'].apply(lambda x: False if x == 9.952 or x == 10.404 else True)

# (6) 특정 컬럼에서의 비정상 패턴 발견
test['Is_Abnormal_WorkMode'] = (test['WorkMode Collect Result_Fill2'] == 3)
test['Pressure_Below_75'] = (test['3rd Pressure Unit Time_AutoClave'] < 75)
test['Suffix_Dam_AJX75334503'] = (test['Model.Suffix_Dam'] == 'AJX75334503')

## **3-E. int로 되어있지만 사실은 범주형 -> object형으로 바꾸는 과정**

In [ ]:
test[['WorkMode Collect Result_Fill2','WorkMode Collect Result_Fill1']] = test[['WorkMode Collect Result_Fill2','WorkMode Collect Result_Fill1']].astype('object')

## **3-F. 랜덤포레스트 학습을 위한 원핫인코딩**

In [ ]:
test = test.drop('Set ID',axis=1)

# step1. 범주형 자료만 뽑기 (target 제외)
categorical_cols = test.select_dtypes(include='object').columns

# step2. 원핫 인코딩
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = onehot_encoder.fit_transform(test[categorical_cols])
encoded_df = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(categorical_cols))

# step3. 나머지 데이터프레임과 concat
# 원핫 인코딩된 데이터와 target 열을 분리
test_non_categorical = test.drop(categorical_cols,axis=1)

# step4. 원핫 인코딩된 데이터프레임과 나머지 데이터프레임을 합치기
test = pd.concat([test_non_categorical, encoded_df], axis=1)

## **3-G. 특정 조건, 특정 값에서 항상 비정상을 보이는 것들 추가**

In [ ]:
conditions = [
    ('Chamber Temp. Unit Time_AutoClave', 183),
    ('Chamber Temp. Unit Time_AutoClave', 180),
    ('Workorder_Dam_3KPXX094-0001', 1.0),
    ('Workorder_Dam_4CPXX084-0001', 1.0)
]
# 각 조건에 대해 새로운 열 추가
for col, value in conditions:
    condition_name = f'Condition_{col}_{value}'
    test[condition_name] = (test[col] == value).astype(int)

# **4. train과 test에서 중복되는 열만 선택**
- train과 test의 일관성 유지

In [296]:
# 두 데이터프레임의 열 이름 추출
columns_train = train.columns
columns_test = test.columns
target = train['target']

# 중복 열 찾기
common_columns = columns_train.intersection(columns_test)

# 중복 열로 새로운 데이터프레임 생성
train = pd.concat([train[common_columns],target],axis=1)
test = test[common_columns]

# **5. Hyper Parameters 수정**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 준비
X = train.drop('target', axis=1)
y = train['target']

# 하이퍼파라미터 그리드 정의
param_grid = {
    'criterion': ['entropy','gini'],
    'n_estimators': [400,500],  # 트리의 수
    'max_depth': [None], # 트리의 최대 깊이
    'min_samples_split': [2], # 내부 노드를 분할하는 데 필요한 최소 샘플 수
    'min_samples_leaf': [1]   # 리프 노드의 최소 샘플 수
}

# RandomForestClassifier 객체 생성
rf = RandomForestClassifier(random_state=110)

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=2)

# 모델 학습
grid_search.fit(X, y)

# 최적의 하이퍼파라미터와 성능 추출
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best F1 score:", best_score)

# **6. 학습**
- Imbalance 데이터이기 때문에 언더샘플링 진행
- **하지만 언더샘플링 특성상 다수의 클래스의 정보손실이 생긴다. 그래서 언더 샘플링을 무작위로 100번정도 진행하여 100개의 학습된 랜덤포레스트를 만든 후 평균을 취하는 방식으로 모델을 합친다.**

In [297]:
X = train.drop('target',axis=1)
y = train['target']

# 데이터 분할 (train/validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForestClassifier의 하이퍼파라미터 설정
rf_params = {
    'criterion': 'gini',
    'n_estimators': 400,  # 트리의 수
    'max_depth': None, # 트리의 최대 깊이
    'min_samples_split': 2, # 내부 노드를 분할하는 데 필요한 최소 샘플 수
    'min_samples_leaf': 1,   # 리프 노드의 최소 샘플 수
    'random_state': 110
}

# 모델 설정
n_undersampling_iterations = 100

# 개별 RandomForestClassifier 모델을 학습시키기 위한 리스트
rf_models = []

# 무작위 언더샘플링 및 모델 학습
for _ in range(n_undersampling_iterations):
    # 비정상 데이터만 샘플링
    df_abnormal = X_train[X_train["target"] == 1]
    df_normal = X_train[X_train["target"] == 0]
    
    # 비정상 데이터의 수만큼 샘플링
    n_abnormal_sample = int(len(df_abnormal))
    df_abnormal_sampled = df_abnormal.sample(n=n_abnormal_sample, replace=False)
    
    # 정상 데이터도 비정상 데이터 수에 맞게 샘플링
    df_normal_sampled = df_normal.sample(n=n_abnormal_sample, replace=False)
    
    # 샘플링된 데이터 결합
    df_balanced = pd.concat([df_abnormal_sampled, df_normal_sampled])
    
    # 훈련 데이터와 타겟 분리
    X_balanced = df_balanced.drop('target', axis=1)
    y_balanced = df_balanced['target']
    
    # 랜덤포레스트 모델 학습
    rf_model = RandomForestClassifier(**rf_params)
    rf_model.fit(X_balanced, y_balanced)
    rf_models.append(rf_model)

# 예측 수행 및 성능 평가
def predict_with_multiple_models(models, X):
    """ 여러 모델의 예측을 평균하여 최종 예측을 결정합니다. """
    predictions = np.zeros((X.shape[0], len(models)))
    for i, model in enumerate(models):
        predictions[:, i] = model.predict_proba(X)[:, 1]
    return (np.mean(predictions, axis=1) > 0.5).astype(int)

# 검증 데이터에 대해 예측 수행
y_pred_val = predict_with_multiple_models(rf_models, X_val)

# f1 score 계산
f1_val = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score: {f1_val:.4f}")

Validation F1 Score: 0.2891


# **7. 제출**

In [283]:
dct = {0:'Normal',1:'AbNormal'}
prediction = pd.Series(predict_with_multiple_models(rf_models, test)).map(dct)

df_sub = pd.read_csv("submission.csv")
df_sub["target"] = prediction

# 제출 파일 저장
df_sub.to_csv("연구7_배깅100_2.csv", index=False)